# Netflix EDA

## Importing Libraries and Dataset

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandasai import SmartDataframe
from pandasai.llm import OpenAI
from dotenv import load_dotenv

load_dotenv()

sns.set_style()


In [ ]:
netflix_titles_df = pd.read_csv('netflix_titles.csv', encoding='iso-8859-1')
api_token = os.getenv('OPEN_AI_API_TOKEN')
llm = OpenAI(api_token=api_token)
data = SmartDataframe(netflix_titles_df, config={ "llm": llm, "verbose": True })

data.chat("what is the shape of the data")

In [ ]:
methods = dir(data)
print(methods)

In [ ]:
data.chat("what are some emergent patterns that you notice in the data?")

In [ ]:
data.chat("Provide more details about these emergent patterns")